In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [34]:
def _salaryDayInMonth(today, delta = 0):
    ### Returns the day in which salary payments take place in that month
    # general salary date in Sweden is the 25th, except
    # if 25th is a saturday, payment is on 24th
    # if 25th is a sunday, payment is on 26th
    
    # Support the option to get the result for the previous month, which is needed
    # when the current day is before the salary payment of that month
    # e.g. on the 12th of Mar, the salary was on the 25th of Feb
    if not (delta == 0 or delta == -1):
        raise Exception("Unsupported value for delta give")
    
    # If the previous month is selected for a today date in January, that is december in the previous year
    deltaYear = 0
    if today.month == 1 and delta == -1:
        deltaYear = -1
        delta = 11
    
    newDate = dt.datetime(today.year+deltaYear, today.month+delta, 25)
    
    if newDate.weekday() == 5:
        newDate = newDate.replace(day=24)
        # return 24
    elif newDate.weekday() == 6:
        newDate = newDate.replace(day=26)
        # return 26
        
    return newDate
    
def daysSinceSalary(today):
    ### Calculate the days since the last salary payment
    salary = _salaryDayInMonth(today)
    
    delta = today - today # small trick to make "delta.days" work if "if" and "elif" statement are both not executed
    
    if today.day > salary.day:
        delta = today - salary
    elif today.day < salary.day:
        prevSalary = _salaryDayInMonth(today, delta=-1)
        delta = today - prevSalary
        
    return delta.days

In [35]:
def processDate(df):
    # Convert into basic year, month, day and weekday
    df["Date"] = pd.to_datetime(df["Date"], format = "%Y-%m-%d")
    
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Weekday"] = df["Date"].dt.weekday
    
    # Calculate days since last salary
    df["DaysSinceSalary"] = df["Date"].apply(lambda dt: daysSinceSalary(dt))
    
    # Determine season
    # df["Season"] = df["Date"].apply(lambda dt: season(dt))
    
    # Include weather data
    # df["Temp..."]
    
    # Drop date column as the model should not use it
    # df = df.drop(["Date"], axis = 1)
    
    return df

In [28]:
def season(today):
    # No clear implementation for season in Sweden yet
    return 

In [29]:
def addMissingDates(df, date_range):
    ### Add NaN sales for dates that do not exist in current range
    ### These values will later be filled
    
    # Loop over the three companies
    for i in range(3):
        # Find dates for which no values are given
        missingDates = date_range.difference(df[df["Company"] == i]["Date"])
        
        # Create a new dataframe, set the dates, sales and respective company
        missingDatesDf = pd.DataFrame({"Date": missingDates})
        missingDatesDf["Sales"] = None
        missingDatesDf["Company"] = i
        
        # Add the new data to the main dataframe
        df = pd.concat([df, missingDatesDf])
    
    return df

In [32]:
# The date range of the data
salesRange = pd.date_range(start = "2020-01-01", end = "2023-01-04")

# File names
historical_set = "caspecoHistoricalData.csv"
predict_set = "caspecoTestRange.csv"
processed_set = "caspecoHistoricalDataProcessed.csv" # this file will include all the processed features



hist_df = pd.read_csv(historical_set)
# test_df = pd.read_csv(test_set)

hist_df = addMissingDates(hist_df, date_range = salesRange) # add missing dates

hist_df = processDate(hist_df) # feature engineer with respect to date
# test_df = processDate(test_df)

# Sort the dataset, by date and by company
hist_df.sort_values(by = ["Date", "Company"], inplace = True)

hist_x = hist_df.loc[:, hist_df.columns != "Sales"]
hist_y = hist_df.loc[:, hist_df.columns == "Sales"]

hist_df.to_csv(processed_set, index=False)

# Just to get an overview of what the data currently looks like
print(hist_x.head())
print(hist_y.head())

           Date  Company  Year  Month  Day  Weekday  DaysSinceSalary
0    2020-01-01        0  2020      1    1        2                7
0    2020-01-01        1  2020      1    1        2                7
3084 2020-01-01        2  2020      1    1        2                7
3083 2020-01-02        0  2020      1    2        3                8
3082 2020-01-02        1  2020      1    2        3                8
             Sales
0              NaN
0              NaN
3084  72608.623649
3083  25421.830339
3082  57957.790779


In [33]:
missingSales = hist_x[hist_y["Sales"].isnull()]

print(f"There are missing values for {len(missingSales['Date'].unique())} dates\n")
# Missing values for 
for i in range(2):
    print(f"For company {i} the following dates are missing")
    print(missingSales[missingSales["Company"] == i]["Date"])

There are missing values for 194 dates

For company 0 the following dates are missing
0     2020-01-01
1     2020-04-12
2     2020-06-19
3     2020-06-20
4     2020-06-21
         ...    
105   2022-09-30
106   2022-12-13
107   2022-12-24
108   2022-12-26
109   2023-01-01
Name: Date, Length: 110, dtype: datetime64[ns]
For company 1 the following dates are missing
0     2020-01-01
1     2020-06-19
2     2020-08-03
3     2020-08-04
4     2020-08-05
         ...    
100   2022-07-06
101   2022-07-07
102   2022-07-08
103   2022-12-24
104   2022-12-31
Name: Date, Length: 105, dtype: datetime64[ns]
